In [33]:
import pandas as pd
import os
from main import get_miner
from log_parsing import from_id_to_template
import ast
from datetime import datetime

In [34]:
df = pd.read_csv(f"output/features_labeled_v4.csv")

In [35]:
df.head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause
0,2022-08-22 03:54,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '23_106', 'score': 0.5400894187779434, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_100'}, {'events': '24_100', 'score': 0.5384615384615384, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '24_99'}, {'events': '28_208', 'score': 0.5315315315315315, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_120'}, {'events': '104_100', 'score': 0.518854415274463, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_106'}, {'events': '38_78', 'score': 0.5053908355795148, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '38_86'}]","[{'events': '76_4', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197'}, {'events': '104_106', 'score': 0.5638418079096045, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23'}, {'events': '23_100', 'score': 0.52, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20'}, {'events': '120_198', 'score': 0.5140495867768595, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '98_100', 'score': 0.5100250626566416, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",['CpuUsageRate(%)']
1,2022-08-22 03:55,frontend-579b9bff58-t2dbm,2022-08-22 03:53:54,cpu_contention,2022-08-22 03:51,"[{'events': '76_197', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)'}, {'events': '23_100', 'score': 0.5517241379310345, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20'}, {'events': '69_146', 'score': 0.5172413793103449, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn'}, {'events': '69_158', 'score': 0.5172413793103449, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn'}]","[{'events': '76_4', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'resource_alert': 'CpuUsageRate(%)', 'events_actual': '76_197'}, {'events': '104_106', 'score': 0.7028169014084507, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '24_100', 'score': 0.6, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '104_100', 'score': 0.5992865636147443, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '120_198', 'score': 0.5738007380073801, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}]",['CpuUsageRate(%)']
2,2022-08-22 04:03,frontend-579b9bff58-t2dbm,2022-08-22 04:02:07,return,2022-08-22 03:51,"[{'events': '18_191', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '18_86'}, {'events': '4_40', 'score': 0.5735294117647058, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_3'}, {'events': '28_208', 'score': 0.543859649122807, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_120'}, {'events': '4_42', 'score': 0.543859649122807, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_3'}, {'events': '16_41', 'score': 0.543859649122807, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '16_15'}, {'events': '4_18', 'score': 0.5251509054325956, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_3'}, {'events': '76_4', 'score': 0.5183044315992292, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}]","[{'events': '120_198', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '28_119', 'score': 1.0, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resourc

In [36]:
df['result_list_inhealthy_lessoften'] = df['result_list_inhealthy_lessoften'].apply(lambda x: ast.literal_eval(x))
df['result_list_inhealthy_moreoften'] = df['result_list_inhealthy_moreoften'].apply(lambda x: ast.literal_eval(x))
df['root_cause'] = df['root_cause'].apply(lambda x: ast.literal_eval(x))

In [37]:
df['inject_type'].unique()

array(['cpu_contention', 'return', 'cpu_consumed', 'exception',
       'network_delay'], dtype=object)

In [38]:
df['root_cause'].apply(str).unique()

array(["['CpuUsageRate(%)']",
       "['Serving product page started', 'GetProduct start']",
       "['Placing order started', 'Order placed complete']",
       "['NetworkP90(ms)']",
       "['Start charge card', 'Charge successfully']",
       "['Received ad request', 'No context provided']",
       "['Query product with name', 'Query product successfully']"],
      dtype=object)

In [39]:
def get_timediff(inject_time, abnormal_time):
    inject_dt = datetime.strptime(inject_time, '%Y-%m-%d %H:%M:%S')
    abnormal_dt = datetime.strptime(abnormal_time, '%Y-%m-%d %H:%M')
    
    diff = max(1, abs(inject_dt.minute - abnormal_dt.minute))
    return diff

df["inject_comp_tdiff"] = df.apply(lambda row: get_timediff(row['inject_time'], row['abnormal_time']), axis=1)

In [40]:
df.inject_comp_tdiff.value_counts()

1    56
2    56
Name: inject_comp_tdiff, dtype: int64

In [41]:
log_template_miner = get_miner()

In [42]:
def get_nezha_rank(result_list, root_cause, inject_pod):
    rank = 1
    rc_found_flag = False
            
    if len(root_cause) == 1:
        for i in range(len(result_list)):
            if "resource_alert" in result_list[i].keys():
                if str(root_cause[0]) in str(result_list[i]["resource_alert"]) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
            else:
                if "events_actual" in result_list[i].keys() and str(root_cause[0]) in from_id_to_template(int(result_list[i]["events_actual"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                    rc_found_flag = True
                    break
                if i > 0:
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    elif len(root_cause) == 2:
        for i in range(len(result_list)):
            if root_cause[0] in from_id_to_template(int(result_list[i]["events"].split(
                    "_")[0]),log_template_miner) and root_cause[1] in from_id_to_template(int(result_list[i]["events"].split("_")[1]),log_template_miner) and inject_pod in str(result_list[i]["pod"]):
                rc_found_flag = True
                break
            else:
                if i > 0:
                    # logger.info("%s, %s",
                    #             result_list[i-1]["score"], result_list[i]["score"])
                    if result_list[i-1]["score"] == result_list[i]["score"] and result_list[i-1]["deepth"] == result_list[i]["deepth"]:
                        continue
                    else:
                        rank += 1
                elif i == 0:
                    rank += 1
    
    if not rc_found_flag:
        return -1
    return rank

In [49]:
df["nezha_rank_050"] = df.apply(lambda row: get_nezha_rank(row['result_list_inhealthy_moreoften'], row['root_cause'], row['inject_pod']), axis=1)

In [44]:
pd.set_option('display.max_colwidth', None)

In [48]:
mask = (df.inject_comp_tdiff == 2) & (df.nezha_rank_050 != 1)
df.iloc[47:48, :][['result_list_inhealthy_moreoften', 'nezha_rank_050']]

,result_list_inhealthy_moreoften,nezha_rank_050
47,"[{'events': '16_41', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '104_106', 'score': 0.5870588235294117, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23'}, {'events': '17_16', 'score': 0.5714285714285714, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '17_190'}, {'events': '38_84', 'score': 0.5446735395189003, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_86'}, {'events': '4_18', 'score': 0.5221238938053098, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '38_78', 'score': 0.5154494382022472, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_86'}, {'events': '69_146', 'score': 0.5121951219512195, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': '69_68'}, {'events': '69_158', 'score': 0.5121951219512195, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': '69_68'}, {'events': '120_198', 'score': 0.5090016366612111, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '98_100', 'score': 0.5062189054726368, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}]",-1


In [46]:
52/56, 54/56

(0.9285714285714286, 0.9642857142857143)

In [52]:
mask = df.inject_comp_tdiff == 2
df[mask].nezha_rank_050.value_counts()

 1    50
-1     4
 2     2
Name: nezha_rank_050, dtype: int64

In [ ]:
47/56, 49/56

(0.8392857142857143, 0.875)

In [56]:
mask = (df.nezha_rank_050 == -1) & (df.inject_comp_tdiff == 2 )
pd.set_option('display.max_colwidth', 1000)
df[mask][['abnormal_time', 'result_list_inhealthy_moreoften', 'result_list_inhealthy_lessoften', 'nezha_rank_050']]

,abnormal_time,result_list_inhealthy_moreoften,result_list_inhealthy_lessoften,nezha_rank_050
47,2022-08-22 07:55,"[{'events': '16_41', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '104_106', 'score': 0.5870588235294117, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '104_23'}, {'events': '17_16', 'score': 0.5714285714285714, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '17_190'}, {'events': '38_84', 'score': 0.5446735395189003, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_86'}, {'events': '4_18', 'score': 0.5221238938053098, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '38_78', 'score': 0.5154494382022472, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_86'}, {'events': '69_146', 'score': 0.5121951219512195, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': '69_68'}, {'events': '69_158', 'score': 0.5121951219512195, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn', 'events_actual': '69_68'}, {'events': '...","[{'events': '17_190', 'score': 1.0, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '17_88'}, {'events': '23_100', 'score': 0.6976744186046512, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20'}, {'events': '4_40', 'score': 0.5797101449275363, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_38'}, {'events': '24_100', 'score': 0.5714285714285714, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '24_99'}, {'events': '4_50', 'score': 0.55, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_38'}, {'events': '4_42', 'score': 0.5357142857142857, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_38'}, {'events': '28_120', 'score': 0.5283018867924528, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_208'}, {'events': '23_106', 'score': 0.5228818800247371, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-...",-1
83,2022-08-23 14:32,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '23_100', 'score': 0.5686274509803921, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '4_40', 'score': 0.5522388059701493, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '4_18', 'score': 0.5436241610738255, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '16_15', 'score': 0.5436241610738255, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '120_198', 'score': 0.5387263339070568, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '76_4', 'score': 0.5302226935312832, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '23_106', 'score': 0.521819299323909, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}]","[{'events': '69_146', 'score': 0.5533980582524272, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn'}, {'events': '69_158', 'score': 0.5533980582524272, 'deepth': 5, 'pod': 'checkoutservice-578fcf4766-9csqn'}, {'events': '4_50', 'score': 0.5384615384615384, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '38_84', 'score': 0.5084459459459459, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '38_86'}, {'events': '4_3', 'score': 0.5, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}]",-1
103,2022-08-23 16:07,"[{'events': '24_100', 'score': 0.7, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '23_100', 'score': 0.6744186046511628, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '4_50', 'score': 0.6, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_38'}, {'events': '4_18', 'score': 0.5388026607538803, 'deepth': 1, 'pod'

In [ ]:
df.columns

Index(['abnormal_time', 'inject_pod', 'inject_time', 'inject_type',
       'normal_time', 'result_list_inhealthy_lessoften',
       'result_list_inhealthy_moreoften', 'root_cause', 'inject_comp_tdiff',
       'nezha_rank_050'],
      dtype='object')

In [ ]:
df.to_csv(f"output/features_labeled_4llm_v4.csv", index=False)